In [ ]:
%pip install --quiet --upgrade diffusers transformers accelerate invisible_watermark mediapy

In [ ]:
use_refiner = False

In [ ]:
import mediapy as media
import random
import sys
import torch

from diffusers import DiffusionPipeline

pipe = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16",
    )

if use_refiner:
  refiner = DiffusionPipeline.from_pretrained(
      "stabilityai/stable-diffusion-xl-refiner-1.0",
      text_encoder_2=pipe.text_encoder_2,
      vae=pipe.vae,
      torch_dtype=torch.float16,
      use_safetensors=True,
      variant="fp16",
  )

  refiner = refiner.to("cuda")

  pipe.enable_model_cpu_offload()
else:
  pipe = pipe.to("cuda")

In [ ]:
prompt_str = "Minimalistic room with wooden flooring, white walls. Quality wood bed, muted white nightstand. Organized wardrobe with 10 jeans, four shoe pairs. A bicycle, laptop for work, and small wooden table."
seed = random.randint(0, sys.maxsize)

def generate_images(prompt: str):
  images = pipe(
      prompt = prompt,
      output_type = "latent" if use_refiner else "pil",
      generator = torch.Generator("cuda").manual_seed(seed),
      ).images

  if use_refiner:
    images = refiner(
        prompt = prompt,
        image = images,
        ).images
  return images

images = generate_images(prompt_str)

print(f"Prompt:\t{prompt_str}\nSeed:\t{seed}")
media.show_images(images)
images[0].save("output.jpg")


In [ ]:
!pip install flask ngrok flask-ngrok  pyngrok

In [ ]:
from PIL import Image
import io

def image_to_byte_array(image: Image):
  imgByteArr = io.BytesIO()
  image.save(imgByteArr, format=image.format)
  imgByteArr = imgByteArr.getvalue()
  return imgByteArr

In [ ]:
from flask import Flask, jsonify
from threading import Timer
from flask_ngrok import run_with_ngrok
from pyngrok import ngrok, conf
from flask import request, send_file
import io
from PIL import Image
from flask import current_app
import json
import requests

app = Flask(__name__)
token = ""# Generate your token at https://dashboard.ngrok.com/ and add it here
conf.get_default().auth_token = token

@app.route('/hello', methods=['GET'])
def hello():
    return jsonify(message="hello")

@app.route('/get_image', methods=['GET'])
def test_image():
    return send_file('pule.jpg', mimetype='image/jpg')
    # return current_app.send_static_file('output.jpg')


@app.route('/get_image_json', methods=['GET'])
def get_image_json():
  tunnel_url = requests.get("http://localhost:4040/api/tunnels").text
  di = json.loads(tunnel_url)
  public_url = di["tunnels"][0]["public_url"]
  out_dict = {"created": 123, "data": [{"url": "{}/get_image".format(public_url)}]}
  return jsonify(out_dict)
  # json.loads('{ "created": 1694880584, "data": [ { "url": "https://en.m.wikipedia.org/wiki/File:Cute_dog.jpg" } ] }')
    # return send_file('output.jpg', mimetype='image/jpg')
    # return current_app.send_static_file('output.jpg')



@app.route('/generate_image', methods=['GET', 'POST'])
def generate_image():
    data = request.get_json()
    prompt = data['prompt']
    print(prompt)
    images = generate_images(prompt)
    images[-1].save("pule.jpg")
    return send_file('pule.jpg', mimetype='image/jpg')


def stop():
    app.stop()



In [ ]:
# run for 8 hours (28800 seconds) then stop
run_with_ngrok(app)

t = Timer(28800.0, stop)
t.start()

app.run()